In [1]:
import pandas as pd
import numpy as np

In [38]:
df = pd.read_csv("./original_data/final_medicine_inventory_sales_data.csv")
df.head(100)

,SKU,Product Name,Category,Dosage Form,Strength,Package Size,Batch Number,Manufacturing Date,Expiration Date,Unit Cost,...,Delivery Date,Transaction Date,Season,Holiday,Promotion,Lag_1,Lag_2,Lag_3,Moving Average,Demand Forecast
0,SKU_1,Medicine_SKU_1,Antifungal,Syrup,231mg,331ml,BN8255,2021-11-23,2022-12-20,14.33,...,2023-01-27,2023-01-31,Winter,0,0,62,62,62,62.0,186
1,SKU_2,Medicine_SKU_2,Antifungal,Tablet,170mg,357ml,BN9857,2021-11-10,2023-04-25,42.15,...,2023-02-08,2023-01-31,Winter,1,0,43,43,43,43.0,129
2,SKU_3,Medicine_SKU_3,Antiviral,Tablet,123mg,117ml,BN5909,2022-02-04,2024-11-19,24.47,...,2023-01-25,2023-01-31,Winter,0,1,61,61,61,61.0,183
3,SKU_4,Medicine_SKU_4,Antiviral,Tablet,263mg,312ml,BN8825,2021-12-24,2023-01-02,13.89,...,2023-01-27,2023-01-31,Winter,0,0,71,71,71,71.0,213
4,SKU_5,Medicine_SKU_5,Antiviral,Tablet,475mg,204ml,BN4163,2022-06-24,2025-03-17,10.16,...,2023-02-04,2023-01-31,Winter,0,1,26,26,26,26.0,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SKU_96,Medicine_SKU_96,Antiviral,Tablet,432mg,375ml,BN8569,2021-11-20,2023-07-27,14.99,...,2023-02-05,2023-01-31,Winter,0,1,47,47,47,47.0,141
96,SKU_97,Medicine_SKU_97,Antibiotic,Injection,152mg,317ml,BN8548,2021-11-27,2023-12-04,21.50,...,2023-01-19,2023-01-31,Winter,0,0,18,18,18,18.0,54
97,SKU_98,Medicine_SKU_98,Painkiller,Tablet,356mg,283ml,BN2724,2022-02-17,2023-12-23,16.63,...,2023-02-09,2023-01-31,Winter,0,0,35,35,35,35.0,105
98,SKU_99,Medicine_SKU_99,Painkiller,Capsule,81mg,145ml,BN8078,2022-07-08,2023-10-01,37.78,...,2023-02-07,2023-01-31,Winter,0,0,47,47,47,47.0,141


In [39]:
df.columns

Index(['SKU', 'Product Name', 'Category', 'Dosage Form', 'Strength',
       'Package Size', 'Batch Number', 'Manufacturing Date', 'Expiration Date',
       'Unit Cost', 'Selling Price', 'Stock In', 'Stock Out', 'Stock Balance',
       'Distributor ID', 'Distributor Name', 'Country', 'Distributor Type',
       'Order Date', 'Quantity Ordered', 'Total Order Cost', 'Order Status',
       'Payment Status', 'Shipping Date', 'Delivery Date', 'Transaction Date',
       'Season', 'Holiday', 'Promotion', 'Lag_1', 'Lag_2', 'Lag_3',
       'Moving Average', 'Demand Forecast'],
      dtype='object')

In [40]:
df['Category'].value_counts()

Category
Antibiotic    916
Painkiller    909
Antifungal    898
Antiviral     877
Name: count, dtype: int64

In [41]:
#converting date columns to datetime 

df['Manufacturing Date'] = pd.to_datetime(df['Manufacturing Date'])
df['Expiration Date'] = pd.to_datetime(df['Expiration Date'])
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Delivery Date'] = pd.to_datetime(df['Delivery Date'])
df['Shipping Date'] = pd.to_datetime(df['Shipping Date'])

In [42]:
#encoding sku column 

from sklearn.preprocessing import LabelEncoder

Le = LabelEncoder()
df['SKU'] = Le.fit_transform(df['SKU'])

In [43]:
# converting season column to numerical features 
df['Season'] = df['Season'].map ({'Spring': 0, 'Summer': 1, 'Autumn': 2, 'Winter': 3})

In [44]:
#Feature Engineering 
df['Days_Until_Expiry'] = (df['Expiration Date'] - pd.to_datetime('today')).dt.days 
df['Lead_Time'] = (df['Delivery Date'] - df['Order Date']).dt.days

In [45]:
# Select relevant columns
final_columns = [
        'SKU','Stock In', 'Stock Out','Stock Balance', 'Days_Until_Expiry', 'Lead_Time',
        'Season', 'Holiday', 'Promotion', 'Lag_1', 'Lag_2', 'Lag_3', 
        'Moving Average', 'Demand Forecast'
    ]

In [46]:
df = df[final_columns]
print (df)

      SKU  Stock In  Stock Out  Stock Balance  Days_Until_Expiry  Lead_Time  \
0       0       186         62            124               -827         11   
1      62       129         43             86               -701         12   
2      73       183         61            122               -127          6   
3      84       213         71            142               -814          7   
4      95        78         26             52                 -9          8   
...   ...       ...        ...            ...                ...        ...   
3595   52         0         87             35                328          9   
3596   53         0        100           -207                731          9   
3597   54         0         98             76                183         10   
3598   55         0         82           -194                174          8   
3599   57         0         59             76                591         11   

      Season  Holiday  Promotion  Lag_1  Lag_2  Lag

In [49]:
df['Promotion'].dtype

dtype('int64')